# Importar Lib's

In [1]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact
from openpyxl import Workbook

# Passo 1: Listar arquivos PDF em uma pasta específica

In [2]:
diretorio_lote = r"C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\01-HOPPECKE\000_Relatórios de Campo\001_RELATÓRIOS_ENVIADOS\001_CORRETIVAS"
#"C:\Users\HELY-DELL\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\1.0-CORRETIVA\2ªTRANCHE"

In [3]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\jupyter\OEM\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 9), step=1)

## Pasta contendo os arquivos PDF

In [4]:
def encontrar_duplicados(numocorrencia):
    duplicados = numocorrencia[numocorrencia.duplicated(subset='NUMOCORRENCIA')]
    return duplicados

arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

string = pd.DataFrame(pd.Series(arquivos_pdf).str.extract(r"(\d+)_(\d+)_(.+)_(.+)_(.+)_(.+)"))
string.rename(columns={1: 'NUMOCORRENCIA'}, inplace=True)
numocorrence = string[['NUMOCORRENCIA']].astype('float')  # Convertendo para float para acomodar NaN

# Contagem total de duplicatas
duplicadas_count = numocorrence[numocorrence.duplicated(subset='NUMOCORRENCIA')].shape[0]

# Encontrar duplicatas
date_duplicated= encontrar_duplicados(numocorrence.copy())  # Cria uma cópia independente
date_duplicated.dropna(inplace=True)  # Remove NaNs do DataFrame duplicatas
date_duplicated['NUMOCORRENCIA'] = date_duplicated['NUMOCORRENCIA'].astype('int')  # Convertendo de volta para int
# Mesclando os dados principais.
df_pdf = numocorrence.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
df_pdf = df_pdf[['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]
print("Número total de duplicatas:", duplicadas_count)
print("Duplicatas encontradas após remoção de NaN:")
display(df_pdf)

Número total de duplicatas: 0
Duplicatas encontradas após remoção de NaN:


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO


# Lista de arquivos PDF na pasta

# Passo 2: Ler o arquivo Excel com a lista de arquivos

In [5]:
caminho_arquivo_excel = df_filtered_global_oem
caminho_arquivo_excel['NUMOCORRENCIA'].count()
df_excel = caminho_arquivo_excel [['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

In [6]:
df_excel

,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
16,202400030922,NaN,42604,NaN,JOAO NUNES DA SILVA,53.0,VALIDADO,ONEL007,2024-05-05 17:40:00
93,202400028348,NaN,18369,NaN,LUCINEIDE LIMA MORAIS,54.0,VALIDADO,ONEOM003,2024-04-25 10:41:00
307,202400023547,NaN,18255,NaN,ROZIANE SILVA DE PAIVA,54.0,VALIDADO,ONEOM003,2024-04-08 10:15:00
383,202400021574,NaN,18440,NaN,TANCREDO GOMES DA SILVA,54.0,VALIDADO,ONEOM003,2024-03-25 10:35:00
1034,202400014330,NaN,18421,NaN,ELIANE GADELHA DA SILVA,54.0,VALIDADO,ONEL016,2024-03-01 14:34:00


## DataFrame a partir do arquivo Excel


#  Passo 3: Comparar os DataFrames para identificar arquivos ausentes no PDF e arquivos não listados no Excel

In [7]:
arquivos_ausentes_no_pdf = df_excel[~df_excel['NUMOCORRENCIA'].isin(df_pdf['NUMOCORRENCIA'])]
arquivos_nao_listados_no_excel = df_pdf[~df_pdf['NUMOCORRENCIA'].isin(df_excel['NUMOCORRENCIA'])]

## Resultados

In [8]:
print(arquivos_ausentes_no_pdf['NUMOCORRENCIA'].count())
display(arquivos_ausentes_no_pdf)

0


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
16,202400030922,NaN,42604,NaN,JOAO NUNES DA SILVA,53.0,VALIDADO,ONEL007,2024-05-05 17:40:00
93,202400028348,NaN,18369,NaN,LUCINEIDE LIMA MORAIS,54.0,VALIDADO,ONEOM003,2024-04-25 10:41:00
307,202400023547,NaN,18255,NaN,ROZIANE SILVA DE PAIVA,54.0,VALIDADO,ONEOM003,2024-04-08 10:15:00
383,202400021574,NaN,18440,NaN,TANCREDO GOMES DA SILVA,54.0,VALIDADO,ONEOM003,2024-03-25 10:35:00
1034,202400014330,NaN,18421,NaN,ELIANE GADELHA DA SILVA,54.0,VALIDADO,ONEL016,2024-03-01 14:34:00


In [9]:
arquivos_nao_listados_no_excel['NUMOCORRENCIA'].count()
nao_listado_excel = arquivos_nao_listados_no_excel.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
nao_listado_excel

,NUMOS_x,UC_x,IDSIGFI_x,NUMOCORRENCIA,NOMECLIENTE_x,ROTA_x,STATUS_x,EXECUTOR_x,DTCONCLUSAO_x,Unnamed: 0,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG


In [10]:
recorrence= df_filtered_global_oem.merge(date_duplicated, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')
recorrence['IDSIGFI'] = recorrence['IDSIGFI'].astype('int')

In [11]:
duplicated_numocorrencia = date_duplicated.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')

In [12]:
print("listar arquivo duplicados:")
print(recorrence.shape)
display(recorrence)

print("listar nº ocorrencia que esta duplicada")
display(sorted(duplicated_numocorrencia['NUMOCORRENCIA']))  

print("Arquivos ausentes na lista de PDF:")
print(arquivos_ausentes_no_pdf.shape)
display(arquivos_ausentes_no_pdf)

print("\nArquivos não listados no Excel:")
print(arquivos_nao_listados_no_excel.shape)
display(arquivos_nao_listados_no_excel)

listar arquivo duplicados:
(5, 52)


,Unnamed: 0,ORIGEM,TIPO,NUMOS,NUMOCORRENCIA,ROTA,NOMECLIENTE,CPF,CODIGOENERGISA,UC,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG
0,False,ENERGISA,CORRETIVA,202400030922,NaN,53.0,JOAO NUNES DA SILVA,95718222215,1,NaN,...,23,2024-05-05 17:04:00,2024-05-05 17:04:00,2024-05-04 01:02:00,3,2024-05-05,202400030922,Report2HOPPECKE,1500,NaN
1,False,ENERGISA,CORRETIVA,202400028348,NaN,54.0,LUCINEIDE LIMA MORAIS,79445730259,L2.2023-0899,NaN,...,24,2024-04-25 09:41:00,2024-04-25 09:41:00,2024-04-24 18:03:00,3,2024-04-25,202400028348,Report2HOPPECKE,2666,NaN
2,False,ENERGISA,CORRETIVA,202400023547,NaN,54.0,ROZIANE SILVA DE PAIVA,038.468.032-19,L2.2023-0785,NaN,...,24,2024-04-08 08:46:00,2024-04-08 08:46:00,2024-04-05 21:42:00,3,2024-04-08,202400023547,Report2HOPPECKE,2666,NaN
3,False,ENERGISA,CORRETIVA,202400021574,NaN,54.0,TANCREDO GOMES DA SILVA,794.453.732-00,L2.2023-0970,NaN,...,21,2024-03-25 09:49:00,2024-03-25 09:49:00,2024-03-25 09:46:00,3,2024-03-25,603920240325094016,Report2HOPPECKE,2666,NaN
4,False,ENERGISA,CORRETIVA,202400014330,NaN,54.0,ELIANE GADELHA DA SILVA,66569729268,L2.2023-0951,NaN,...,24,2024-03-01 11:47:00,2024-03-01 11:47:00,2024-03-01 11:45:00,3,2024-03-01,607820240301113984,Report2HOPPECKE,2666,NaN


listar nº ocorrencia que esta duplicada


[]

Arquivos ausentes na lista de PDF:
(5, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
16,202400030922,NaN,42604,NaN,JOAO NUNES DA SILVA,53.0,VALIDADO,ONEL007,2024-05-05 17:40:00
93,202400028348,NaN,18369,NaN,LUCINEIDE LIMA MORAIS,54.0,VALIDADO,ONEOM003,2024-04-25 10:41:00
307,202400023547,NaN,18255,NaN,ROZIANE SILVA DE PAIVA,54.0,VALIDADO,ONEOM003,2024-04-08 10:15:00
383,202400021574,NaN,18440,NaN,TANCREDO GOMES DA SILVA,54.0,VALIDADO,ONEOM003,2024-03-25 10:35:00
1034,202400014330,NaN,18421,NaN,ELIANE GADELHA DA SILVA,54.0,VALIDADO,ONEL016,2024-03-01 14:34:00



Arquivos não listados no Excel:
(0, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO


In [13]:
#Salvando a planilha Manutenções Energisa_preventiva para ser enviadas em Excell( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\OEM\output\pdf_ausentes_corretivas_3_TRANCHE-geral.xlsx")
arquivos_ausentes_no_pdf.to_excel(dataframe_excel)
dataframe_excel.close()